# Applied Neural Networks - Exercises 

**NOTICE:**
1. You are allowed to work in groups of up to three people but **have to document** your group's\
 members in the top cell of your notebook.
2. **Comment your code**, explain what you do (refer to the slides). It will help you understand the topics\
 and help me understand your thinking progress. Quality of comments will be graded. 
3. **Discuss** and analyze your results, **write-down your learnings**. These exercises are no programming\
 exercises it is about learning and getting a touch for these methods. Such questions might be asked in the\
 final exams. 
 4. Feel free to **experiment** with these methods. Change parameters think about improvements, write down\
 what you learned. This is not only about collecting points for the final grade, it is about understanding\
  the methods. 

### Exercise 1 - Data Normalization and Standardization


**Summary:** In this exercise you will implement the min-max normalization and standardization and compare it to\
sklearn's implementation. It is important to remember, that we always normalize or standardize for all samples\
 over a single feature dimension.


**Provided Code:** In the cell below I have provided you with a sample code to initialize some dummy data.\
The parameter ```n_samples``` defines the number of samples we have in the training set (the number of $x_i$)\
while ```n_features``` defines the number of dimensions of each sample feature vector. 


**Your Tasks in this exercise:**
1. Implement the MinMax Normalization and Standardization. 
2. Use the ```MinMaxScaler``` and ```StandardScaler``` from sklearn to verify your results. 


In [170]:
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler

x,y = make_regression(n_samples=10, n_features=5)

### Exercise 2 - Softmax

**Summary:** In this exercise you will implement the softmax activation using the naive and numerically\
more stable log-sum variation. 


**Provided Code:** In the cell below there is some sample code that generates sample inputs. 


**Your Tasks in this exercise:**
1. Implement the softmax function using the naive approach. 
2. Implement the softmax function using the log-sum trick. 
3. Compare your two implementations for numerical stability\
(experiment with different values of std) and verify
your results using ```tf.nn.softmax```



In [8]:
import numpy as np 
import tensorflow as tf

mu = 0
std = 10
xi = mu + std * np.random.randn(10)